In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import sys
effdet_path = "../input/effdet"
sys.path.append(effdet_path)
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"
sys.path.append(timm_path)
import timm
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
omega_path = "../input/omegaconf"
sys.path.append(omega_path)
from omegaconf import OmegaConf
import glob
import sklearn
import math
import random

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim
from torchvision import transforms

from transformers import get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics, model_selection, preprocessing
from sklearn.model_selection import GroupKFold
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
!pip install --no-deps '../input/pycocotools202/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl' > /dev/null

# Model

In [ ]:
IMAGE_SIZE = 320
from effdet import create_model

In [ ]:
model = create_model("tf_efficientdet_lite0" , bench_task='predict' , num_classes= 1 , image_size=(IMAGE_SIZE,IMAGE_SIZE),pretrained=False,pretrained_backbone=False)
path = "../input/efficientdet-train/Fold 0 model with val loss 1.4488916648517955.pth"
model.load_state_dict(torch.load(path))
model.eval()
model.to(device)

# Prediction

In [ ]:
test_transform = A.Compose([
   A.Resize(IMAGE_SIZE,IMAGE_SIZE ,p = 1.0),
            A.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
            ToTensorV2()
])

In [ ]:
def format_prediction_string(boxes, scores):
    # Format as specified in the evaluation page
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.2f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
prediction_confidence_threshold = 0.15
def predict(model, pixel_array,prediction_confidence_threshold):
    # Predictions for a single image
    
    # Apply all the transformations that are required
    pixel_array = np.array(pixel_array, dtype=np.float32)
    image = test_transform(image=pixel_array)['image'].unsqueeze(0)
    
    # Get predictions
    #each row representing [x_min, y_min, x_max, y_max, score, class]
    with torch.no_grad():
        detections = model(image.to(device)) 
    
    # Move predictions to cpu and numpy
    
    boxes = detections[0].detach().cpu().numpy()[:, :4]
    scores = detections[0].detach().cpu().numpy()[:, 4]
    indexes = np.where(scores > prediction_confidence_threshold)[0]
    boxes = boxes[indexes]
    scores =scores[indexes]
    
    boxes[:, 0] = (boxes[:, 0]*(1280/IMAGE_SIZE)).astype(np.int32).clip(min=0, max=1280)
    boxes[:, 1] = (boxes[:, 1]*(720/IMAGE_SIZE)).astype(np.int32).clip(min=0, max=720)
    boxes[:, 2] = (boxes[:, 2]*(1280/IMAGE_SIZE)).astype(np.int32).clip(min=0, max=1280)
    boxes[:, 3] = (boxes[:, 3]*(720/IMAGE_SIZE)).astype(np.int32).clip(min=0, max=720)
    
    
    # Go back from x_min, y_min, x_max, y_max to x_min, y_min, w, h
    boxes[:, 2] = (boxes[:, 2] - boxes[:, 0])
    boxes[:, 3] = (boxes[:, 3] - boxes[:, 1])
    
    
  
    # Format results as requested in the Evaluation tab
    return format_prediction_string(boxes, scores)

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

In [ ]:
for (pixel_array, df_pred) in iter_test:  # iterate through all test set images
    df_pred['annotations'] = predict(model, pixel_array,prediction_confidence_threshold)
    env.predict(df_pred)